<a href="https://colab.research.google.com/github/Riddars/Dream_team/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22Copy_of_%D0%A5%D0%B0%D0%BA%D0%B0%D1%82%D0%BE%D0%BD_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install shap

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import shap
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [ ]:

# Загрузка данных из CSV файлов
data_df = pd.read_csv('https://raw.githubusercontent.com/Riddars/Hakaton/main/data.csv', index_col=0)
bacterial_descriptors_df = pd.read_csv('https://raw.githubusercontent.com/Riddars/Hakaton/main/bacterial_descriptors.csv')
drug_descriptors_df = pd.read_csv('https://raw.githubusercontent.com/Riddars/Hakaton/main/drug_descriptors.csv', index_col=0)

Смотрим шапки датасетов

Была предпринята попытка хаполнить пропуски в данных. Окончательным успехом она не увенчаалсь. В данных очень много пропусков.
Есть два метода MIC, MBC почему-то они не требуют знания концентрации NP и дозы лекарства. (~25% data)


Для нужных бактерий нужно найти какие они грам-положительные или грам-отрицательные. Это важно по статье которую нашла Настя, ну и из общих соображений "очевидно"

# Основной датасет

In [ ]:
data_df.head(2)

In [ ]:
data_df.info()

In [ ]:
# в каких колонках есть пропуски

data_df.isna().sum(), data_df.isna().sum()/len(data_df)*100

In [ ]:
## for preprocessing results


def data_df_fix_column_types(df=data_df):
    start_len = len(data_df)
    data_df["shape"] = data_df["shape"].astype("category")
    data_df["method"] = data_df["method"].astype("category")

    # 32+ насколько оно больше непонятно, при этом из всего 10 - можно пожертвовать
    # Escherichia coli	, Actinobacillus pleuropneumoniae
    # chem_synthesis_reduction_by_D_maltose
    data_df.drop(
        data_df[data_df.ZOI_drug_NP == '32+'].index, inplace=True)
    data_df.drop(
        data_df[data_df.ZOI_drug == '32+'].index, inplace=True)
    data_df['ZOI_drug'] = data_df.ZOI_drug.astype(float)

    # 50+ замечено 7 штук
    data_df.drop(
        data_df[data_df.ZOI_NP == '50+'].index,inplace=True)
    # # 4000 <- перед ними последнее значение это 220. Из всего 3
    # data_df.drop(
    #     data_df[data_df.ZOI_NP == '4000'].index,inplace=True)
    data_df["ZOI_NP"] = data_df.ZOI_NP.astype(float)


    # 17+2 <- это вероятно 17 +/- 2
    # data_df.drop(
    #     data_df[data_df.ZOI_drug_NP == '17+2'].index, inplace=True)
    data_df["ZOI_drug_NP"] = data_df["ZOI_drug_NP"].replace('17+2', 17)
    data_df["ZOI_drug_NP"] = data_df.ZOI_drug_NP.astype(float)

    # NP_concentration 170/85/*
    '''
    если посмотреть на:
    mask_ = (data_df.NP_concentration.str.contains("170/85/")) | (data_df.NP_concentration == '42.5')
    data_df[mask_].sort_values("Bacteria").set_index("Bacteria")
    такое преобразование выглядит разымным
    '''
    data_df["NP_concentration"] = (data_df["NP_concentration"]
                                        .apply(lambda x : float(str(x).split('/')[-1]))
                                )
    finish_len = len(data_df)
    print("Было удалено :", start_len - finish_len, "строк")


data_df_fix_column_types(data_df)

In [ ]:
# эксперименты только с наночастицами
mask1_ = (data_df.ZOI_drug.isna()) & (data_df.ZOI_drug_NP.isna()) & (~data_df.ZOI_NP.isna()) & (~data_df.NP_concentration.isna())
data_df.loc[mask1_, 'Drug_dose'] = 0
data_df.loc[mask1_, ['Drug', 'Drug_class_drug_bank']] = ['No', 'No']
data_df.loc[mask1_, "ZOI_drug"] = 0
data_df.loc[mask1_, "ZOI_drug_NP"] = data_df.loc[mask1_, "ZOI_NP"]

# эксперименты только с лекарствами
mask2_ = (data_df.ZOI_NP.isna()) & (data_df.ZOI_drug_NP.isna()) & (~data_df.ZOI_drug.isna()) & (~data_df.Drug_dose.isna())
data_df.loc[mask2_] # а тут ноль экземпляров

In [ ]:
data_df.isna().sum()

## изучаем датасет бактерий


Вспомогательный датасет 2 (бактерии)

In [ ]:
bacterial_descriptors_df.head(2)

In [ ]:
# смотрим каких названий из data_df нет в bacterial_descriptors
set(data_df['Bacteria'].unique()) - set(bacterial_descriptors_df['Bacteria'].unique())

# а вот избыточных много
# set(bacterial_descriptors_df['Bacteria'].unique()) - set(data_df['Bacteria'].unique())

In [ ]:
data_df.replace('Bacillus spp.        ', 'Bacillus sp.', inplace=True)
data_df.replace('Acinetobacter baumanii', 'Acinetobacter baumannii', inplace=True)
data_df.replace('Actinobacillus pleuropneumoniae ', 'Actinobacillus pleuropneumoniae', inplace=True)
# data_df.replace('Candida glochares', 'Candida glochares', inplace=True) # ??? заменить само на себя?

data_df.replace('Enterobacter cloacae ', 'Enterobacter cloacae', inplace=True)
data_df.replace('Salmonella  typhi', 'Salmonella typhi', inplace=True)
data_df.replace('Salmonella Paratyphi', 'Salmonella enterica', inplace=True) # https://www.ncbi.nlm.nih.gov/datasets/taxonomy/54388/
# data_df.replace('Serratia marcescens', 'Serratia marcescens', inplace=True)  # ??? заменить само на себя?


# unrecoverable
data_df.drop(
    data_df[
        (data_df.Bacteria == 'Serratia marcescens') | ((data_df.Bacteria == 'Candida glochares'))
    ].index,
    inplace=True
)

In [ ]:
# fix some grams
bacterial_descriptors_df[bacterial_descriptors_df.gram.isna()].Bacteria.value_counts()

bacterial_descriptors_df.loc[
    bacterial_descriptors_df.Bacteria.isin(
        ['Candida albicans',
         'Candida glabrata',
         'Candida saitoana',
         'Micrococcus luteus',
        'Staphylococcus aureus'
        'Staphylococcus epidermidis',
         'Bacillus subtilis',
         'Bacillus sp.',
         ])
, 'gram'
] = 'p'

bacterial_descriptors_df.loc[
    bacterial_descriptors_df.Bacteria.isin(
[
            'Neisseria mucosa',
            'Escherichia coli',
            'Salmonella enterica',
            'Salmonella typhi',
            'Klebsiella pneumonia',
            'Pseudomonas aeruginosa',
            'Vibrio parahaemolyticus',
            'Serratia odorifera',
            'Pseudomonas koreensis',
            'Haemophilus influenzae',
         ])
, 'gram'
] = 'n'


Вспомогательный датасет 3 (антибиотики)

In [ ]:
drug_descriptors_df.head(2)

Смотрим описание основного датасета

In [ ]:
data_df.info()

объединяем дф основной и дф бактерий и дф артибиотиков

In [ ]:
df = data_df.merge(bacterial_descriptors_df, how='left', on='Bacteria')

In [ ]:
df = df.merge(drug_descriptors_df, how='left', left_on='Drug', right_on='drug')

# set(df['Drug'].unique()) - set(df['Drug'].unique()), df["Drug"].value_counts()["Neomycin"]

df.drop(
    df[df.Drug == 'Neomycin'].index, inplace= True
)

# Feature инжиринг

In [ ]:
import networkx as nx

bacteria_taxonomy_descriptors = [
    'kingdom', 'subkingdom', 'clade',
    'phylum', 'class', 'order', 'family', 'genus', 'Bacteria']
# добавить расстояния на графе таксономии
def make_graph_better(bacterial_descriptors_df=bacterial_descriptors_df):
    '''
    Строит филогенетический граф
    '''
    tax_df = bacterial_descriptors_df.loc[:,bacteria_taxonomy_descriptors] # копирует
    gg_ = nx.Graph()
    for top_node in tax_df['kingdom']: # чтобы был связным
        gg_.add_edge('root', top_node)

    for parents, childs in zip(bacteria_taxonomy_descriptors, bacteria_taxonomy_descriptors[1:]):
        tax_df[childs] = tax_df.loc[:, [childs]].fillna(childs) # уникальный NaN для кадждого столбца
        edge_lst = tax_df[[parents, childs]].values
        gg_.add_edges_from(edge_lst)

    # gg_.remove_node(np.nan)
    # nx.draw_spring(gg_)
    return gg_


def get_distances(bacterial_descriptors_df, df):
    '''Берёт из графа расстояния и отфильтровывает только нужные
    '''
    g_= make_graph_better(bacterial_descriptors_df)
    lengths = dict(nx.all_pairs_shortest_path_length(g_))
    rv = {key: lengths[key] for key in df.Bacteria.unique()}
    return rv


def populate_distances_to_df(bacterial_descriptors_df=bacterial_descriptors_df,
                             df=df):
    df = df.copy()
    distances = get_distances(bacterial_descriptors_df, df)
    for bacteria in distances.keys():
        df['d_to_' + bacteria] = df.Bacteria.map(distances[bacteria])
    # return df.loc[:, df.columns.str.startswith('d_to_')] # посмотреть на расстояния
    return df

def remove_bacteria_taxonomy_descriptors_from_df(df):
    df.drop(columns=bacteria_taxonomy_descriptors, inplace=True)

df = populate_distances_to_df(bacterial_descriptors_df, df)
remove_bacteria_taxonomy_descriptors_from_df(df)

In [ ]:
def drop_fold_increase_in_antibacterial_activity(df):
    df.drop(columns='fold_increase_in_antibacterial_activity (%)', inplace=True)

drop_fold_increase_in_antibacterial_activity(df)

In [ ]:
def convert_gram_to_valeus(df):
    df['gram'] = df.gram.map({'p': 1, 'n': -1})
    # .astype(int)

convert_gram_to_valeus(df)

### попытка нарисовать филогенетический граф с количеством семплов на каждый вид

In [ ]:
# # как нарисовать дерево https://stackoverflow.com/questions/57512155/how-to-draw-a-tree-more-beautifully-in-networkx
# # вариант 2: https://plotly.com/python/tree-plots/
# tree_ = make_graph_better()
# leafes = [x for x in tree_.nodes() if tree_.degree(x) == 1]
# weights = df.Bacteria.value_counts()
# for l in leafes:
#     tree_.nodes[l]['weight'] = weights[l]

# # nx.draw_circular(tree_)

Выкидываем первый столбик

In [ ]:
df = df.iloc[:, 1:]

Убираем лишние столбцы

In [ ]:
# наоборот - какие колонки выбросить

df = df.drop(
    columns=[
#  'Drug',
 'Drug_class_drug_bank',
#  'Drug_dose',
#  'NP_concentration',
#  'NP size_min',
#  'NP size_max',
#  'NP size_avg',
#  'shape',
#  'method',
#  'ZOI_drug',
#  'ZOI_NP',
#  'ZOI_drug_NP',
#  'fold_increase_in_antibacterial_activity (%)',
#  'MDR_check',
   'Tax_id',
#  'kingdom',
#  'subkingdom',
#  'clade',
#  'phylum',
#  'class',
#  'order',
#  'family',
#  'genus',
#  'species',
#  'gram',
#  'min_Incub_period, h',
#  'avg_Incub_period, h',
#  'max_Incub_period, h',
#  'growth_temp, C',
#  'biosafety_level',
 'isolated_from',
 'drug',
 'chemID',
 'prefered_name',
 'smiles'
 ]
    )

# df.head()

In [ ]:
df.info()

In [ ]:
#importing all the libraries needed
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

sns.heatmap(df[['NP size_min', 'NP size_avg', 'NP size_max']].corr(), annot = False, xticklabels=True)

# data cleaning

In [ ]:
# # приведение всех значений столбца к типу string
# data_df['ваш столбец'] = data_df['ваш столбец'].astype(str)

# # Проверка наличия отсутствующих значений
# missing_values = data_df.isnull().sum()
# # Удаление строк с хотя бы одним пропущенным значением
# data_df.dropna(inplace=True)

# # Посмотреть кол-во дубликатов в столбце
# duplicates_count = data_df['ваш столбец'].duplicated().sum()
# print('Количество дубликатов:', duplicates_count)
# # удаление дубликатов
# data_df.drop_duplicates(subset='ваш столбец', keep='first', inplace=True)

# # Подсчет количества уникальных элементов в таблице
# unique_count = df['Столбец'].nunique()
# print('Количество уникальных элементов:', unique_count)

# # Удаление ненужных столбцов
# df = df.drop(columns=['ваш столбец'])

# # Заполнение пустых значений
# df = df.fillna('')

# # Удаление строк с выбросами
# data = data[(data['NP size_avg'] >= 0) & (data['NP size_avg'] <= 50)]

# # Исправление опечаток
# data['Bacteria'] = data['Bacteria'].str.replace('Escherichia coli', 'E. coli')

# # Фильтрация по условию
# filtered_data = data[data['Drug_class_drug_bank'] == 'Lactams']

# УДАЛЕНИЕ ВЫБРОСОВ

In [ ]:
# #удалим выбросы по интерквартельному размаху
# def IQR_data_clean(df, colname):
#     filter = df[colname]
#     Q1, Q3 = filter.quantile(0.25), filter.quantile(0.75)
#     IQR = Q3 - Q1
#     dfr = df[
#         ~(
#             (filter < (Q1 - 1.5 * IQR)) |(filter > (Q3 + 1.5 * IQR))
#         )
#     ]
#     return dfr

In [ ]:
cat = r'''
 /\_/\
( o.o )
 > ^ <
'''

print(cat)

In [ ]:
# data_df.NP_Synthesis.unique()

# Anastasiia's


In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# f, axes = plt.subplots(3, 3)
# sns.distplot(df['Drug_dose'], kde=True, ax = axes[0,0])
# sns.distplot(df['NP size_min'], kde=True, ax = axes[0,1])

# plt.show()

# DF2 bacterial_descriptors_df

In [ ]:
bacterial_descriptors_df[['gram', 'species']].drop_duplicates()

# Функция split - разделение на тест и трейн

In [ ]:
#пока сделаем OneHot
df = df.drop(columns = ['ZOI_NP'])
condition = ((df['method'] == 'disc_diffusion') | (df['method'] == 'well_diffusion'))
df1 = df[condition]
df2 = df[~condition]
df1=df1.dropna()
df5=pd.get_dummies(df1)
df5.info()

In [ ]:
target = "ZOI_drug_NP"
def split(df, target):
  X_train, X_test, y_train, y_test = train_test_split(df.drop(columns = target,axis=1), df[target], test_size=0.3, random_state=42)
  cols = df.columns.tolist()
  columns_to_remove = [target]
  cols = list(filter(lambda x: x not in columns_to_remove, cols))
  from sklearn.preprocessing import MinMaxScaler
  scaler = MinMaxScaler()
  sclr = scaler.fit(X_train)
  X_train = pd.DataFrame(scaler.transform(X_train), columns=cols)
  X_test = pd.DataFrame(scaler.transform(X_test), columns=cols)
  return (X_train, X_test, y_train, y_test)
X_train, X_test, y_train, y_test = split(df5,target)

# Функция Random Forest

In [ ]:
from sklearn.metrics import  mean_squared_error
def RF(X_train, y_train, X_test, y_test):
  regr = RandomForestRegressor(random_state=0, n_estimators=174, max_depth=15, oob_score = True)
  regr.fit(X_train, y_train)
  explainer = shap.TreeExplainer(regr)
  shap_values = explainer(X_train)
  predicted = regr.predict(X_test)

  fig, ax = plt.subplots()
  ax.scatter(y_test, predicted, edgecolors=(0, 0, 0))
  ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)
  ax.set_xlabel('Measured')
  ax.set_ylabel('Predicted')
  plt.show()

  mse = mean_squared_error(y_test, predicted)
  rmse = np.sqrt(mse)

   # Рассчет RMSE в процентах
  rmse_percentage = (rmse / np.mean(y_test)) * 100

  # Вывод результатов
  print("RMSE:", rmse)
  print("RMSE в процентах:", rmse_percentage)
  print(r2_score(y_test, predicted))
  shap.summary_plot(shap_values, X_train, plot_type='bar')
  return(regr)

In [ ]:
r = RF(X_train, y_train, X_test, y_test)

In [ ]:
df1.info()

# Что пытался делать **Арсен**

In [ ]:
# df = pd.read_csv('https://raw.githubusercontent.com/Riddars/Hakaton/main/Final(no).csv', index_col=0)

In [ ]:
# import pandas as pd
# from sklearn.preprocessing import StandardScaler
# from sklearn.impute import SimpleImputer
# from sklearn.model_selection import train_test_split
# # Удаление строк с пропущенными значениями
# df.dropna(inplace=True)
# # Разделение данных на признаки (X) и целевую переменную (y)
# X = df[['NP_concentration', 'NP size_min', 'NP size_max', 'ZOI_drug', 'MDR_check']]
# y = df['ZOI_drug_NP']
# # Шкалирование признаков с использованием стандартного масштабирования
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)
# # Импьютация пропущенных значений с использованием средних значений
# imputer = SimpleImputer(strategy='mean')
# X_imputed = imputer.fit_transform(X_scaled)

# # Разделение данных на обучающий и тестовый наборы
# X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

In [ ]:
# # Импорт необходимых библиотек
# from sklearn.linear_model import LinearRegression
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error
# from sklearn.metrics import r2_score
# import matplotlib.pyplot as plt
# import numpy as np

# # Разделение данных на признаки (X) и целевую переменную (y)
# X = df[['NP_concentration', 'NP size_min', 'NP size_max', 'ZOI_drug', 'MDR_check']]
# y = df['ZOI_drug_NP']

# # Разделение данных на обучающий и тестовый наборы
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Создание и обучение модели линейной регрессии
# model = LinearRegression()
# model.fit(X_train, y_train)

# # Предсказание на тестовом наборе
# y_pred = model.predict(X_test)

# # Вычисление среднеквадратичной ошибки (MSE)
# mse = mean_squared_error(y_test, y_pred)
# print('Среднеквадратичная ошибка:', mse)

# # Рассчитываем коэффициент детерминации
# r2 = r2_score(y_test, y_pred)
# print('Коэффициент детерминации (R^2):', r2)

# # Визуализация рассеяния фактических значений и предсказанных значений
# plt.scatter(y_test, y_pred)
# plt.xlabel('Фактические значения')
# plt.ylabel('Предсказанные значения')
# plt.title('Рассеяние: Фактические значения vs Предсказанные значения')
# plt.show()

# # Визуализация линии регрессии
# plt.plot(y_test, y_test, color='red', label='Идеальное соответствие')
# plt.plot(y_test, y_pred, 'o', color='blue', label='Линия регрессии')
# plt.xlabel('Фактические значения')
# plt.ylabel('Предсказанные значения')
# plt.title('Линия регрессии: Фактические значения vs Предсказанные значения')
# plt.legend()
# plt.show()

In [ ]:
# import tensorflow as tf
# from tensorflow import keras
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# import matplotlib.pyplot as plt

# # Загрузка данных из CSV файла
# data = df

# # Разделение на признаки и целевую переменную
# X = data.drop('ZOI_drug_NP', axis=1)
# y = data['ZOI_drug_NP']

# # Масштабирование признаков
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(df.drop(columns=["species", "genus", "family", "order", "class", "kingdom", "phylum", "ZOI_drug_NP"]))

# # Разделение на обучающую и тестовую выборки
# X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# # Создание модели нейронной сети
# model = keras.Sequential([
#     keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
#     keras.layers.Dropout(0.2),
#     keras.layers.Dense(64, activation='relu'),
#     keras.layers.BatchNormalization(),
#     keras.layers.Dense(1)
# ])

# # Компиляция модели с метрикой accuracy
# model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# # Обучение модели
# history = model.fit(X_train, y_train, epochs=100, validation_split=0.2)

# # Оценка модели на тестовой выборке
# loss, accuracy = model.evaluate(X_test, y_test)

# # Вывод результатов
# print('Test Loss:', loss)
# print('Test Accuracy:', accuracy)

# # Построение графика обучения
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Model Loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Validation'], loc='upper right')
# plt.show()

In [ ]:
# from sklearn.preprocessing import OneHotEncoder

# # Создание экземпляра OneHotEncoder
# encoder = OneHotEncoder(sparse=False)

# # Преобразование категориальных признаков методом OneHotEncoding
# encoded_features = encoder.fit_transform(df[categorical_features])

# # Получение имен категорий
# feature_names = encoder.get_feature_names_out(categorical_features)

# # Создание датафрейма из преобразованных признаков с префиксом "encoded_"
# encoded_df = pd.DataFrame(encoded_features, columns=[f'encoded_{name}' for name in feature_names])

# # Переименование столбцов в encoded_df (если необходимо)
# encoded_df = encoded_df.rename(columns={'old_name': 'new_name'})

# # Объединение преобразованных признаков с исходным датафреймом
# df = pd.concat([df, encoded_df])

# # Удаление исходных категориальных признаков
# df = df.drop(columns=categorical_features)